In [2]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"

# train test split paths
train_path = data_path / 'tts/train'
test_path = data_path / 'tts/test'
val_path = data_path / 'tts/val'

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# silence max image size warning
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 

# import modeling packages
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# modeling metrics
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

changes to data generators:
    
    train batch_size 20

In [2]:
train_generator = ImageDataGenerator().flow_from_directory(str(train_path),
                                                           target_size=(150, 150),
                                                           batch_size=20,
                                                           class_mode='categorical')
test_generator = ImageDataGenerator().flow_from_directory(str(test_path),
                                                          target_size=(150, 150),
                                                          batch_size=50,
                                                          class_mode='categorical',
                                                          shuffle=False)
val_generator = ImageDataGenerator().flow_from_directory(str(val_path),
                                                          target_size=(150, 150),
                                                          batch_size=109,
                                                          class_mode='categorical',
                                                          shuffle=False)

Found 4000 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
Found 545 images belonging to 5 classes.


changes to model architecture:
        
    1st conv2d num_filters 30 -> 35
    
    1st dense layer dropout .25 -> .2
    
    dense layers 2

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(35, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(30, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 35)      980       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 35)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 30)        9480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 30)        0         
_________________________________________________________________
flatten (Flatten)            (None, 38880)             0         
_________________________________________________________________
dense (Dense)                (None, 40)                1555240   
_________________________________________________________________
dropout (Dropout)            (None, 40)                0

changes to fit:
    
    epochs 6 -> 10

In [5]:
model.fit(
        train_generator,
        steps_per_epoch=200,
        epochs=10,
        validation_data=test_generator,
        validation_steps=10)

Epoch 1/10
 58/200 [=======>......................] - ETA: 5:25 - loss: 12.3313 - accuracy: 0.2086

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


200/200 [==============================] - 522s 3s/step - loss: 4.7215 - accuracy: 0.2247 - val_loss: 1.5981 - val_accuracy: 0.2820
Epoch 2/10
200/200 [==============================] - 528s 3s/step - loss: 1.5481 - accuracy: 0.3243 - val_loss: 1.6099 - val_accuracy: 0.2500
Epoch 3/10
200/200 [==============================] - 512s 3s/step - loss: 1.4373 - accuracy: 0.3947 - val_loss: 1.6359 - val_accuracy: 0.3080
Epoch 4/10
200/200 [==============================] - 519s 3s/step - loss: 1.2965 - accuracy: 0.4695 - val_loss: 1.8014 - val_accuracy: 0.3060
Epoch 5/10
200/200 [==============================] - 544s 3s/step - loss: 1.1243 - accuracy: 0.5627 - val_loss: 1.9168 - val_accuracy: 0.3280
Epoch 6/10
200/200 [==============================] - 522s 3s/step - loss: 0.9891 - accuracy: 0.6302 - val_loss: 1.8772 - val_accuracy: 0.3140
Epoch 7/10
200/200 [==============================] - 516s 3s/step - loss: 0.9189 - accuracy: 0.6725 - val_loss: 2.1655 - val_accuracy: 0.3220
Epoch 8/10

In [1]:
conf_mat = confusion_matrix(test_generator.classes, np.argmax(model.predict(test_generator), axis=1))

class_names = list(test_generator.class_indices.keys())

fig, ax = plt.subplots(figsize=(8,6))
model_7_path = pathlib.Path().absolute().parent.parent / 'reports/figures/mod_8_heatmap.png'
ax.set_ylim([0,1])
sns.heatmap(conf_mat, xticklabels=class_names, yticklabels=class_names, ax=ax, square=True, annot=True, cmap='Blues')
plt.savefig(model_7_path)

NameError: name 'confusion_matrix' is not defined

In [8]:
accuracies = []
for i in range(0, 5):
    acc = conf_mat[i][i] / sum(conf_mat[i])
    accuracies.append(acc)
accuracies

[0.2, 0.24, 0.61, 0.31, 0.27]